In [1]:
from transformers import EsmTokenizer, EsmForMaskedLM
import torch




In [5]:
# Load MSA-capable model
model_name = "facebook/esm1v_t33_650M_UR90S_1"  # MSA-specific model
tokenizer = EsmTokenizer.from_pretrained(model_name)
model = EsmForMaskedLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

/home/yining_yang/anaconda3/envs/prosst/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of EsmForMaskedLM were not initialized from the model checkpoint at facebook/esm1v_t33_650M_UR90S_1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Example MSA (in real use, you would load this from a file)
msa = [
    "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG",
    "MKTIRQERLKSIVRILERAKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNVIATPRGYVLAGG",
    "MKPVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVLVQDIAYLRSLGYNIVATPRGYVLAGG"
]

# Mask a position in the first sequence
mask_pos = 5
msa[0] = msa[0][:mask_pos] + tokenizer.mask_token + msa[0][mask_pos+1:]

# Tokenize the MSA
msa_encodings = tokenizer(msa, is_split_into_words=False, return_tensors="pt", padding=True)

# Get predictions
with torch.no_grad():
    outputs = model(**msa_encodings)
    logits = outputs.logits

# Get predictions for the masked token in the first sequence
mask_token_index = torch.where(msa_encodings["input_ids"][0] == tokenizer.mask_token_id)[0]
predicted_logits = logits[0, mask_token_index]



In [9]:
predicted_logits.size()

torch.Size([1, 33])

In [ ]:
# Get top predictions
top_tokens = torch.topk(predicted_logits, k=5, dim=1)
for token, score in zip(top_tokens.indices[0], top_tokens.values[0]):
    predicted_token = tokenizer.convert_ids_to_tokens(token)
    print(f"{predicted_token}: {score:.4f}")